## Timbr Machine Usage and Status Inspection

Timbr Machine is a data ingestion and computation tool that can be programmed for linear-like data-pipeline processing. It was be run with juno-magic remotely. A running machine comprises many dynamic processes, and some kind of introspection into the state of the machine is necessary tool for understanding its behavior. This notebook uses a test-case machine instance to demonstrate a basic implementation of a status inspection and discusses the structure used to maintain state information.

In [1]:
from timbr.machine import Machine

### A test-case: streaming twitter as a source

Our test case puts tweets onto the machine queue at a high rate. We included a function that throws an arbitrary error every 31 tweets as a particular model test-case that might be expected to inform the structure and presentation of the machine status.

In [2]:
from twython import TwythonStreamer
import collections
from threading import Thread

from __future__ import print_function
from textblob import TextBlob

MACHINE = Machine()
MACHINE.start()

def raise_error_every_now_and_then(x):
    if recent_data._count > 1 and recent_data._count % 31 == 0:
        raise TypeError

MACHINE[0] = lambda x: x.get("text", "")
MACHINE[1] = lambda x: TextBlob(x).sentiment
MACHINE[2] = lambda a, b: recent_data.append((b, a))
MACHINE[3] = raise_error_every_now_and_then

class CountingDeque(collections.deque):
    def __init__(self, *args, **kwargs):
        self._count = 0
        super(CountingDeque, self).__init__(*args, **kwargs)
    
    def append(self, *args):
        super(CountingDeque, self).append(*args)
        self._count += 1        
        
recent_data = CountingDeque(maxlen=50)
#thatsdogs
app_key = "is4Leas6P8ajv4ERNojyJ7psg"
app_secret = "JIb2EEGWbE6NmS4NbdMCARfoCONdYxwG6mfnLY9Z61Q9ZkM9cD"
access_token = "1881035263-ghn9BPqkY4PMyVdfsuaNEeTYBtRXwfKo8Op07Cw"
token_secret = "uKrTloEChEQWShUfU3FS9ejXyi906HjHbsB4T4QDtE7HW"

class Streamer(TwythonStreamer):
    
    def on_success(self, data):
        #print data
        MACHINE.put(data)
        
#     def on_error(self, status_code, data):
#         MACHINE.put({"error": status_code})
try:
    streamer.disconnect()
    del streamer
except Exception:
    pass
streamer = Streamer(app_key, app_secret, access_token, token_secret)
streamthread = Thread(target= streamer.statuses.filter, kwargs={"track": "trump"})
streamthread.start()

359

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/Users/jamiepolackwich1/anaconda/envs/juno-machine/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/jamiepolackwich1/anaconda/envs/juno-machine/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/Users/jamiepolackwich1/anaconda/envs/juno-machine/lib/python2.7/site-packages/twython/streaming/types.py", line 66, in filter
    self.streamer._request(url, 'POST', params=params)
  File "/Users/jamiepolackwich1/anaconda/envs/juno-machine/lib/python2.7/site-packages/twython/streaming/api.py", line 141, in _request
    for line in response.iter_lines(self.chunk_size):
  File "/Users/jamiepolackwich1/anaconda/envs/juno-machine/lib/python2.7/site-packages/requests-2.10.0-py2.7.egg/requests/models.py", line 706, in iter_lines
    for chunk in self.iter_content(chunk_size=chunk_size, decode_unicode=decode_unicode):
  File "/Users/jam

Admittedtly, that's an unexpected error, and I'm just going to leave this here because it's exactly the kind of thing that's useful for this notebook. 

### Checking machine status

In the meantime, we can check if the machine is still running by checking the running property on the machine instance:

In [8]:
MACHINE.running

True

This property checks whether the actual thread that's running the data consumption is still active. The MachineConsumer which runs there takes data off of a queue and submits it to a dask task graph running synchronously. 

In [3]:
MACHINE.stop()
MACHINE.running

False

In [4]:
try:
    streamer.disconnect()
    del streamer
except Exception:
    pass

State information is maintained in a dictionary as a class attribute created on initiation:

In [ ]:
class BaseMachine(object):
    def __init__(self, ...)
    ...
    self._status = {"LastOID": None, "Processed": 0, "Errored": [], "QueueSize": self.q.qsize()}


The structure maintains a current data consumption ID, the ID of the previously consumed data, the number of data consumed by the machine, a list of errros and the current size of the queue the consumer gets from.

These data are updated in real time with each put and get of the pipline process. Status is updated on every MachineConsumer get, which waits on the actual dask graph to process, and records information about the data consumed. Total data processed works like a counter, even when errors are processed (talk about shortly).

Notably, infomrmation about errors raised in the dask are not presented here, but they are recorded in the status strcuture when they occur, along with the function that raised them:

In [7]:
MACHINE.status

{'Errored': [{'f3': {'err': 'TypeError()',
    'errtime': '2016-07-20T14:06:25+00:00',
    'oid': '578f8561f351e181e43c4324'}},
  {'f3': {'err': 'TypeError()',
    'errtime': '2016-07-20T14:06:27+00:00',
    'oid': '578f8563f351e181e43c4343'}},
  {'f3': {'err': 'TypeError()',
    'errtime': '2016-07-20T14:06:30+00:00',
    'oid': '578f8566f351e181e43c4362'}},
  {'f3': {'err': 'TypeError()',
    'errtime': '2016-07-20T14:06:32+00:00',
    'oid': '578f8568f351e181e43c4381'}},
  {'f3': {'err': 'TypeError()',
    'errtime': '2016-07-20T14:06:34+00:00',
    'oid': '578f856af351e181e43c43a0'}},
  {'f3': {'err': 'TypeError()',
    'errtime': '2016-07-20T14:06:36+00:00',
    'oid': '578f856cf351e181e43c43bf'}}],
 'LastOID': '578f856df351e181e43c43c4',
 'LastProcessedTime': '2016-07-20T14:06:37+00:00',
 'Processed': 190,
 'QueueSize': 0}

Registering a raised in the status structure is the responsibility of a custom class that every user programmed machine function gets "wrapped" in before committed to the machine dask configuration:

In [ ]:
class MachineTransform(object):
    def __init__(self, machine, fn, pos):
        self.machine = machine
        self.fn = wrap_transform(fn)
        self.ref = "f{}".format(pos)

    def on_exception(self, e):
        self.machine._status['Errored'].append({self.ref: {"oid": self.machine._status["LastOID"], "err": e.__repr__(), 
            "errtime": time_from_objectidstr(self.machine._status["LastOID"])}})

    def on_success(self):
        pass

    def __call__(self, *args, **kwargs):
        try:
            return self.fn(*args, **kwargs)
            self.on_success()
        except Exception as e:
            self.on_exception(e)

    def __repr__(self):
        pass

The error information message gets appended to the Machine.status error list with function name, data id, error time (derived from data id), and the string representation of the error (not the actual error), as a dictionary structure as seen above.

### Displaying status

Our display mechanism is simple and builds HTML to be displayed using IPython.display's HTML method. 

A small display of this information is available via Machine.display_status(): 

In [9]:
MACHINE.display_status()

In [ ]:
def display_status(status):
    stats = MACHINE.status
    s0 = "<div style='border:1px; border-style:solid; width:400px; height:auto; float:left;'><b>Current Ingest Time -- {}</b></div>".format(stats['CurrentTime'])
    s1 = "<div style='border:1px; border-style:solid; width:400px; height:auto; float:left;'><b>Current Ingest ID -- {}</b></div>".format(stats['CurrentOID'])
    s2 = "<div style='border:1px; border-style:solid; width:400px; height:auto; float:left;'><b>Total Datum Processed -- {}</b></div>".format(stats['Processed'])
    s3 = "<div style='border:1px; border-style:solid; width:400px; height:auto; float:left;'><b>Current Queue Depth -- {}</b></div>".format(stats["QueueSize"])
    display(HTML("\n".join([s0, s1, s2, s3])))

Although errors are registered in status, information about them are not included in the status display.

### Discussion

Currently, we can check if the machine consumer thread is running and display general information about the machine. We maintain some state information that updates on every get, but is located in the MachineConsumer class because the data is prepared for serialization there, which the status updates with. 

Obviously there's a lot to do here. Some ideas are described in detail in machine-introspection-testcases.ipynb regarding ways to move forward with the way errors are handled on a macine level as well as potentially better ways to maintain much more information around the individual functions in the dask graph using methods available in the dask api.

Individual display message formats can immediately be improved using a pandas or similar kind of ipython-env friendly dataframe native display. 